In [12]:
import pandas as pd
import numpy as np
import plotly.express as px

# Arrest Pairs Part 1

In [13]:
arrest = pd.read_csv('data-part-3/CAAI coding exercise/arrest.csv', 
                     parse_dates=['arrest_date'])
prisoner = pd.read_csv('data-part-3/CAAI coding exercise/prisoner.csv',
                       parse_dates=['birth_date'])

## 1. How many prisoners?

19390

In [14]:
print('unique prisoners in arrest dataset:', arrest['prisoner_id'].nunique())
print('unique prisoners in prisoner dataset:', prisoner['prisoner_id'].nunique())

unique prisoners in arrest dataset: 19390
unique prisoners in prisoner dataset: 19390


## 2. How many arrests?

32728

In [15]:
print('unique arrests in arrest dataset:', arrest['arrest_id'].nunique())

unique arrests in arrest dataset: 32728


## 3. Time periods

2017-03-23 to 2019-08-15

In [16]:
print(arrest['arrest_date'].min())
print(arrest['arrest_date'].max())
# prisoner

2017-03-23 00:00:00
2019-08-15 00:00:00


## 4. How many arrests per day

37.36

In [17]:
arrests_each_day = arrest.groupby('arrest_date').count()['arrest_id'] # arbitrarily take first column; all cols are identical after this grouping
print(arrests_each_day.mean())

# hist for eyeball check. looks plausible
px.histogram(arrests_each_day)

37.36073059360731


## 5. Proportion with bond > 0

0.915

In [18]:
# confirm that data is properly cleaned 
# specifically, that arrest_id <--> bond_amount is bijective 
# this gets all unique bond_amounts for each arrest_id
bonds_arrest_mappings = arrest.groupby('arrest_id')['bond_amount'].agg(lambda x: list(set(x)))

# confirm bijection (raises error if non-unique mappings)
if not set(bonds_arrest_mappings.map(len).unique()) == {1}:
    raise Exception('Non-unique arrest <--> bond mapping!')

In [19]:
# convert bond_amount to pd.Series with one bond amount per arrest.
unique_bonds = bonds_arrest_mappings.str[0]

# analyze
(unique_bonds > 0).mean()

# confirm correct by doing another way
unique_arrests = arrest.drop_duplicates(subset=['arrest_id'])
(unique_arrests['bond_amount'] > 0).mean()

0.9148435590320215

## 6. Proportion of male arrests
0.791

In [20]:
# map prisoner sex into arrest dataframe
unique_arrests = unique_arrests.merge(right=prisoner,
                                      how='left',
                                      left_on='prisoner_id',
                                      right_on='prisoner_id',
                                      suffixes=('','_prisoner')
                                      )

# calculate proportion
(unique_arrests['sex'] == 'M').mean()

0.7913101931068198

## 7. Recidivist proportion
0.298

In [21]:
num_recidivist_prisoners = (arrest['prisoner_id'].value_counts() > 1).sum()
num_total_prisoners = arrest['prisoner_id'].nunique()

num_recidivist_prisoners / num_total_prisoners

0.298091799896854

In [22]:
# plot arrest count per person to QA
px.histogram(arrest['prisoner_id'].value_counts(),
             histnorm='probability density')

## 8. Num times re-arrested
3.31

In [23]:
arrest_counts = arrest['prisoner_id'].value_counts()

arrest_counts[arrest_counts > 1].mean()

3.307612456747405

## 9. Median age of arrested
29.422

In [24]:
arrest_merged = arrest.merge(right=prisoner,
                             how='left',
                             left_on='prisoner_id',
                             right_on='prisoner_id',
                             suffixes=('','_prisoner')
                             )
arrest_merged['age_on_arrest_(days)'] = (arrest_merged['arrest_date'] - arrest_merged['birth_date']).dt.days
arrest_merged['age_on_arrest_(days)'].median() / 365.25

29.422313483915126